In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os
import pandas as pd

import efficientnet.tfkeras as efn

In [3]:
train_image_path = glob.glob('data\\train\\*\\*.png')
labels = [i.split('\\')[2] for i in train_image_path]
labels

['AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',
 'AD',

In [5]:
labels = [0 if i=='AD' else 1 for i in labels ]
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [6]:
# Step1 使用 tf.data.Dataset.from_tensor_slices 进行加载
image_ds = tf.data.Dataset.from_tensor_slices((train_image_path,labels))

In [7]:
val_count = int(len(labels)*0.2)
train_count = len(labels)-val_count

In [8]:
def load_preprocess_image(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[300,300])
    image = tf.image.random_crop(image,[260,260,3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image,0.2)
    
    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image,label

In [9]:
def load_preprocess_image_test(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[260,260])
    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image,label

In [10]:
# 选取验证集
image_train_ds = image_ds.skip(val_count)
image_val_ds = image_ds.take(val_count)

# Step3 预处理 (预处理函数在下面)
AUTOTUNE = tf.data.experimental.AUTOTUNE
image_train_ds = image_train_ds.map(load_preprocess_image,num_parallel_calls=AUTOTUNE)
image_val_ds = image_val_ds.map(load_preprocess_image_test,num_parallel_calls=AUTOTUNE)

In [11]:
BATCH_SIZE = 32
# shuffle 打乱数据 batch设置 batch size repeat设置迭代次数(迭代2次) test数据集不需要
image_train_ds = image_train_ds.repeat().shuffle(train_count).batch(BATCH_SIZE)
image_train_ds = image_train_ds.prefetch(AUTOTUNE)#预取,GPU，CPU加速

image_val_ds = image_val_ds.batch(BATCH_SIZE)
image_val_ds = image_val_ds.prefetch(AUTOTUNE)#预取

In [110]:
# covn_base = keras.applications.VGG19(weights='imagenet',
#                                            input_shape=(256,256,3),
#                                            include_top=False,
#                                            pooling='avg')
covn_base = efn.EfficientNetB2(weights='imagenet',
                               input_shape=(260,260,3),
                               include_top=False,
                               pooling='avg')

In [12]:
covn_base = efn.EfficientNetB3(weights='imagenet',
                               input_shape=(260,260,3),
                               include_top=False,
                               pooling='avg')

In [163]:
covn_base = efn.EfficientNetB5(weights='imagenet',
                               input_shape=(260,260,3),
                               include_top=False,
                               pooling='avg')

115515392/115515256 [==============================] - 337s 3us/step


In [13]:
covn_base.summary()

Model: "efficientnet-b3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 260, 260, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 130, 130, 40) 1080        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 130, 130, 40) 160         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 130, 130, 40) 0           stem_bn[0][0]                    
____________________________________________________________________________________

In [14]:
model = keras.Sequential()
model.add(covn_base)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(4,activation='sigmoid'))

covn_base.trainable = False #设置权重参数不可训练

In [15]:
#编译
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
             loss = 'sparse_categorical_crossentropy',
             metrics=['acc'])

history = model.fit(
    image_train_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=8,
    validation_data=image_val_ds,
    validation_steps=val_count//BATCH_SIZE
)

Epoch 1/8
37/37 [==============================] - 141s 4s/step - loss: 0.6963 - acc: 0.5600 - val_loss: 0.6924 - val_acc: 1.0000
Epoch 2/8
37/37 [==============================] - 163s 4s/step - loss: 0.6943 - acc: 0.4003 - val_loss: 0.6931 - val_acc: 0.8681
Epoch 3/8
 1/37 [..............................] - 33s 33s/step - loss: 0.6931 - acc: 0.4688 - val_loss: 0.6931 - val_acc: 0.8681


In [16]:
covn_base.trainable=True
len(covn_base.layers)

378

In [17]:
fine_tune_at = -3
for layer in covn_base.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.Adam(0.0005),
             loss = 'sparse_categorical_crossentropy',
             metrics=['acc'])

initial_epochs=8
fine_tune_epochs=8
total_epoch = initial_epochs+fine_tune_epochs

history = model.fit(
    image_train_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=total_epoch,
    initial_epoch=initial_epochs,y
    validation_data=image_val_ds,
    validation_steps=val_count//BATCH_SIZE
)

model.save('mstz_model_EfficientNetB52.h5')

Epoch 9/16
37/37 [==============================] - 173s 5s/step - loss: 0.6170 - acc: 0.7111 - val_loss: 3.1153 - val_acc: 0.0000e+00
Epoch 10/16
37/37 [==============================] - 171s 5s/step - loss: 0.5506 - acc: 0.7694 - val_loss: 1.7815 - val_acc: 0.0000e+00
Epoch 11/16
 1/37 [..............................] - 33s 33s/step - loss: 0.4777 - acc: 0.8125 - val_loss: 1.8008 - val_acc: 0.0000e+00


In [18]:
#加载图片
def load_preprocess_images(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[260,260])
    image = tf.cast(image,tf.float32)
    image = image/255
    image = tf.expand_dims(image,0)
    return image

In [24]:
test_image_path = glob.glob('val/*.png')

# test_image_path.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0][1:]))

images = [load_preprocess_images(image_path) for image_path in test_image_path]

In [25]:
len(images)

1000

In [26]:
image_count = len(images)
values = []
result_dict = {}

for i in range(image_count):
    pred = model.predict(images[i])
    values.append(np.argmax(pred))
    print('.',end='')

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [36]:
#写文件
with open('result_efficientB2.csv','w',encoding='utf-8') as f:
    f.write('image_id,category_id\n')
    [f.write('{0},{1}\n'.format(key.split('\\')[1][:-4], value)) for (key,value) in zip(test_image_path,values)]

In [35]:
values=['NC' for i in values ]